# Actual Draft for the Script to run every Sunday

1. Get top 15 stocks by market cap 
2. Get historical returns for the last X days of those stocks 
3. Cumulate these returns and return a sorted dataframe
4. Gather the top and last 3 
5. Intersect with the attention data (see R code)

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

import datetime

***
## Universe Scraper 
Grab top 20 market cap cryptocurrencies (scraped from CoinMarketCap)

In [5]:
###=== UNIVERSE SCRAPER ===###

url = "https://coinmarketcap.com/"
response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")
table = soup.find("table")

rows = table.find_all("tr")

ticker_list = []

for i, row in enumerate(rows[1:101]): # exclude header row
    if i < 10:
        ticker = row.find("a", {"class": "cmc-link"}).find('p', {'class': 'coin-item-symbol'}).text
    else:
        ticker = row.find("a", {"class": "cmc-link"}).find("span", {"class": "crypto-symbol"}).text

    ticker_list.append(ticker)

In [12]:
###=== CLEANING AND SELECTION OF THE UNIVERSE ===###

no_trades = ['USD', 'DAI', 'WBTC', 'WETH']

for i in range(len(no_trades)):    
    ticker_list = [x for x in ticker_list if no_trades[i] not in x]

top_twenty_mktcap = ticker_list[:20]

In [13]:
top_twenty_mktcap

['BTC',
 'ETH',
 'BNB',
 'XRP',
 'DOGE',
 'ADA',
 'SOL',
 'TRX',
 'MATIC',
 'DOT',
 'LTC',
 'SHIB',
 'BCH',
 'AVAX',
 'TON',
 'XLM',
 'LINK',
 'LEO',
 'UNI',
 'ATOM']

***
## Price Scraper

In [19]:
# Returns for past 14 days
start_date = (datetime.date.today() - datetime.timedelta(14)).strftime('%d-%m-%Y')
end_date = (datetime.date.today()).strftime('%d-%m-%Y')

### With CMC Scraper

In [20]:
from cryptocmd import CmcScraper

# Inital DF with Bitcoin 
scraper = CmcScraper("BTC", start_date, end_date)
df = pd.DataFrame(scraper.get_dataframe(date_as_index=True)['Close']).rename(columns={'Close':'BTC'})

for crypto in top_twenty_mktcap[1:]:
    scraper = CmcScraper(str(crypto), start_date, end_date)
    df = pd.merge(df, pd.DataFrame(scraper.get_dataframe(date_as_index=True)['Close']).rename(columns={'Close':str(crypto)}), left_index=True, right_index=True)
price_df = df[::-1]


***
## Strategy itself

In [21]:
def output_strategy(n_ls, data, output_sort=False):

    # Cumulative Returns for the Past Week
    log_ret_df = np.log(1 + data.pct_change())
    cum_returns = log_ret_df.apply(lambda x: x.sum())

    if output_sort == True:
        print(cum_returns.sort_values(ascending=False))
    

    top_stocks = cum_returns.sort_values(ascending=False).index[:n_ls].values
    bottom_stocks = cum_returns.sort_values(ascending=True).index[:n_ls].values

    print(f'Cryptos to long this week: {top_stocks}')
    print(f'Cryptos to short this week: {bottom_stocks}')


In [22]:
output_strategy(3, price_df, output_sort=True)

SHIB     0.200932
UNI      0.028377
BNB      0.015994
LEO      0.005114
DOGE    -0.000033
BTC     -0.004640
SOL     -0.008409
ETH     -0.012259
AVAX    -0.041372
DOT     -0.042571
ADA     -0.044291
XLM     -0.046666
TRX     -0.058545
ATOM    -0.061314
LINK    -0.066614
BCH     -0.069642
LTC     -0.077904
MATIC   -0.088097
XRP     -0.120337
TON     -0.152451
dtype: float64
Cryptos to long this week: ['SHIB' 'UNI' 'BNB']
Cryptos to short this week: ['TON' 'XRP' 'MATIC']
